<a href="https://colab.research.google.com/github/rafaellccouto/pos_tech_dengue/blob/main/postech_aula_dengue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Postech: Análise de dados da Dengue de 2013 a 2023

In [ ]:
#Analise dados Dengue
#Bibliotecas de manipulação de dados
import pandas as pd
import numpy as np
#Bibliotecas de visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
#Base de dados
df_dengue = pd.read_excel('Base de Dados Dengue.xlsx')

In [ ]:
df_dengue.head()

In [ ]:
df_dengue.tail()

## Explorando a base de dados

In [ ]:
print(f'Tamanho da base de dados: {df_dengue.shape}')

In [ ]:
df_dengue.dtypes

### Existem dados nulos?

In [ ]:
df_dengue.isnull()

In [ ]:
df_dengue.isnull().sum()

In [ ]:
print(f'Quantidade de dados nulos: {df_dengue.isnull().sum().sum()}')

### Informações sobre a base

In [ ]:
df_dengue.info()

### Verificar valores únicos

In [ ]:
df_dengue.nunique()

### Quantidade de pessoas que pegaram dengue por município por ano

In [ ]:
#Criar coluna ano
df_dengue['ano'] = pd.DatetimeIndex(df_dengue['data_infeccoes']).year

In [ ]:
df_dengue.head()

In [ ]:
infeccoes_municipio = df_dengue.groupby(['ano','municipio','uf'])['qtd_infeccoes'].sum().reset_index()

In [ ]:
infeccoes_municipio.head()

In [ ]:
infeccoes_municipio.info()

### Agrupamento por estado e ano

In [ ]:
infeccoes_estado = df_dengue.groupby(['ano','uf'])['qtd_infeccoes'].sum().reset_index()

In [ ]:
infeccoes_estado.head()

In [ ]:
infeccoes_estado.tail()

## Estatítstica descritiva

In [ ]:
infeccoes_estado.describe()

In [ ]:
infeccoes_estado.shape

### Utilizando o boxplot

In [ ]:
# Cria boxplot comparando as infecções por ano
plt.figure(figsize=(10,6))
sns.boxplot(x='ano', y='qtd_infeccoes', data=infeccoes_estado)
# Personalizações do gráfico
plt.title('Distribuição das Infecções por Ano')
plt.yscale('log')  # Usar escala logarítmica para melhor visualização
plt.xlabel('Ano')
plt.ylabel('Quantidade de Infecções')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Filtras os dados para o ano de 2023
infeccoes_2023 = infeccoes_estado[infeccoes_estado['ano'] == 2023]
infeccoes_2023

### Resumo do ano de 2023

In [ ]:
infeccoes_2023.describe()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(y='qtd_infeccoes', data=infeccoes_2023)
plt.title('Infecções em 2023')
plt.xticks
plt.ylabel('Quantidade de Infecções')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# utilizando o gráfico de barras para visualizar as infecções por estado em 2023
plt.figure(figsize=(12,6))  
sns.barplot(x='uf', y='qtd_infeccoes', data=infeccoes_2023)
plt.title('Infecções por Estado em 2023')
plt.xlabel('Estado (UF)')
plt.ylabel('Quantidade de Infecções') 
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7) 
plt.show()

### Ordenar os dados em ordem decrescente do número de infecções 20223

In [ ]:
infeccoes_estado_2023_decrescente = infeccoes_2023.sort_values(by='qtd_infeccoes', ascending=False) 

In [ ]:
infeccoes_estado_2023_decrescente

### Plotando o gráfico ordenado

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x='uf', y='qtd_infeccoes', data=infeccoes_estado_2023_decrescente)
plt.title('Infecções por Estado em 2023 (Ordenado)')  
plt.xlabel('Estado (UF)')
plt.ylabel('Quantidade de Infecções') 
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7) 
plt.show()### Plotando o gráfico ordenado

### Construção de um gráfico de tendências por meio de disperção onde para cada estado há uma regressão linear própria, separado por cores e diferentes estilos de pontilhado e o eixo y em potência de 10

In [ ]:
# Gráfico de dispersão com regressão linear por estado (cada estado com cor e estilo de linha),
# eixo y em escala log10. Usa variáveis já definidas no notebook: infeccoes_estado, ufs, linestyles, palette, correlations_df

fig, ax = plt.subplots(figsize=(14, 8))

# parâmetros para plotagem de valores zero (apenas para visualização; não altera os dados)
min_plot_value = 0.1

for idx, uf in enumerate(ufs):
  df_uf = infeccoes_estado[infeccoes_estado['uf'] == uf]
  if df_uf.empty:
    continue

  x = df_uf['ano'].values
  y = df_uf['qtd_infeccoes'].values

  # ponto de scatter (substitui zeros por um valor pequeno para poder mostrar em escala log)
  y_scatter = np.where(y <= 0, min_plot_value, y)

  color = palette.get(uf, None)
  ls = linestyles[idx % len(linestyles)]

  ax.scatter(x, y_scatter, color=color, s=30, alpha=0.8, edgecolor='k', linewidth=0.2)

  # ajusta regressão linear sobre log10(y + 1) para lidar com zeros e heterocedasticidade
  y_log = np.log10(y + 1)
  slope, intercept = np.polyfit(x, y_log, 1)

  x_line = np.linspace(x.min(), x.max(), 200)
  pred_log = slope * x_line + intercept
  pred_y = 10 ** pred_log - 1
  pred_y_plot = np.where(pred_y <= 0, min_plot_value, pred_y)

  ax.plot(x_line, pred_y_plot, color=color, linestyle=ls, linewidth=2)

  # anota o r de Pearson próximo ao final da linha (se disponível)
  r_row = correlations_df[correlations_df['uf'] == uf]
  if not r_row.empty:
    r_val = float(r_row['pearson_r'].values[0])
    # posiciona o texto ligeiramente à direita do último ano
    ax.text(x_line[-1] + 0.08, pred_y_plot[-1], f"{uf} (r={r_val:.2f})",
        color=color, fontsize=8, va='center')

# configurações do eixo y em escala log base 10
ax.set_yscale('log')
ax.set_title('Tendência de Infecções por Estado (regressão linear por estado) — eixo Y em potência de 10')
ax.set_xlabel('Ano')
ax.set_ylabel('Quantidade de Infecções (escala log10)')
ax.set_xticks(np.arange(infeccoes_estado['ano'].min(), infeccoes_estado['ano'].max() + 1))
ax.grid(which='both', linestyle='--', alpha=0.6)
ax.set_ylim(bottom=min_plot_value, top=infeccoes_estado['qtd_infeccoes'].max() * 1.5)

plt.tight_layout()
plt.show()